In [134]:
using GaussianProcesses, Plots, SumProductNetworks, StatsFuns
import SumProductNetworks.add!

Data set

In [188]:
gr(size = (1024, 768))

Plots.GRBackend()

In [3]:
datapath = "../data/clean/motor.csv"

"../data/clean/motor.csv"

In [7]:
(data, header) = readcsv(datapath, header = true)

([2.4 0.0 1.0 3.7; 2.6 -1.3 1.0 3.7; … ; 55.4 -2.7 3.0 138.0; 57.6 10.7 3.0 138.0], AbstractString["times" "accel" "strata" "v"])

In [8]:
headerDict = Dict(col[2] => col[1] for col in enumerate(header))

Dict{SubString{String},Int64} with 4 entries:
  "v"      => 4
  "times"  => 1
  "accel"  => 2
  "strata" => 3

In [9]:
X = convert(Vector, data[:,headerDict["times"]])
y = convert(Vector, data[:,headerDict["accel"]]);

In [10]:
y /= maximum(y);

In [11]:
N = length(X)

94

In [187]:
scatter(X, y)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## Learn a full GP

In [189]:
mZero = MeanZero()                   #Zero mean function
kern = SE(0.0,0.0)                   #Sqaured exponential kernel (note that hyperparameters are on the log scale)

Type: GaussianProcesses.SEIso, Params: [0.0, 0.0]


In [190]:
logObsNoise = -1.0                        # log standard deviation of observation noise (this is optional)

-1.0

In [191]:
exp(-1)

0.36787944117144233

In [179]:
gp = GP(reshape(X, 1, N), y, MeanZero(), SE(log(5.0),log(1.0)), -1.)       #Fit the GP

GP Exact object:
  Dim = 1
  Number of observations = 94
  Mean function:
    Type: GaussianProcesses.MeanZero, Params: Float64[]
  Kernel:
    Type: GaussianProcesses.SEIso, Params: [1.60944, 0.0]
  Input observations = 
[2.4 2.6 … 55.4 57.6]
  Output observations = [0.0, -0.0173333, -0.036, 0.0, -0.036, -0.036, -0.036, -0.0173333, -0.036, -0.036  …  0.142667, 0.142667, -0.357333, -0.177333, 0.0, 0.142667, -0.196, -0.036, -0.036, 0.142667]
  Variance of observation noise = 0.1353352832366127
  Marginal Log-Likelihood = -41.135

In [192]:
gp.target

-41.1350265676093

In [193]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

In [195]:
scatter(X, y, label = "observations", title = "Full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "full GP with 95% interval")
savefig("../plots/fullGP.png")

## optimize a full GP

In [196]:
optimize!(gp)

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-1.0,1.6094379124341003,0.0]
 * Minimizer: [-1.2282746156641002,1.6156825479816077, ...]
 * Minimum: 3.510034e+01
 * Iterations: 9
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 37
 * Gradient Calls: 37

In [198]:
μ, σ² = predict_y(gp,linspace(minimum(X),maximum(X),100));

In [199]:
scatter(X, y, label = "observations", title = "Marginal LLH optimized full GP with SE Kernel and LLH: $(gp.target)")
plot!(linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "optimized full GP with 95% interval")
savefig("../plots/optimized_fullGP.png")

# Make a SPN with GP leaves

In [22]:
global gID = 0

0

In [23]:
function nextID()
    global gID
    gID += 1
    return gID
end

nextID (generic function with 1 method)

### Make a struct for sum GP nodes

In [106]:
mutable struct GPSumNode <: SumNode{Any}
    id::Int
    parents::Vector{SPNNode}
    children::Vector{SPNNode}
    prior_weights::Vector{Float64}
    posterior_weights::Vector{Float64}

    function GPSumNode(id, split; parents = SPNNode[])
        new(id, parents, SPNNode[], Float64[], Float64[])
    end
end

In [171]:
function add!(parent::GPSumNode, child::SPNNode)
    if !(child in parent.children)
        push!(parent.children, child)
        push!(parent.prior_weights, 1.)
        push!(parent.posterior_weights, 1.)
        push!(child.parents, parent)
        
        parent.prior_weights ./= sum(parent.prior_weights)
        parent.posterior_weights ./= sum(parent.posterior_weights)
    end
    
    @assert sum(parent.prior_weights) ≈ 1. "Weights should sum up to one, sum(w) = $(sum(parent.prior_weights))"
    @assert sum(parent.posterior_weights) ≈ 1. "Weights should sum up to one, sum(w) = $(sum(parent.prior_weights))"
 end

add! (generic function with 7 methods)

### Make a struct for split nodes, i.e. products that split the support

In [24]:
mutable struct FiniteSplitNode <: ProductNode
    id::Int
    parents::Vector{SPNNode}
    children::Vector{SPNNode}
    split::Vector{Float64}

    function FiniteSplitNode(id, split; parents = SPNNode[])
        new(id, parents, SPNNode[], split)
    end
end

In [25]:
function add!(parent::FiniteSplitNode, child::SPNNode)
     if !(child in parent.children)
         push!(parent.children, child)
         push!(child.parents, parent)
     end
 end

add! (generic function with 5 methods)

### Make a struct for leaf nodes with GPs as distributions

In [26]:
mutable struct GPLeaf{T} <: Leaf{Any}
    id::Int
    gp::GaussianProcesses.GPE
    parents::Vector{SPNNode}
    
    function GPLeaf{T}(id, gp) where T <: Any
        new(id, gp)
    end
end

### Construct a SPN with GP leaves using random splits

Hand-coded 2-layer example

In [201]:
srand(12345678)
K = 10;
numSamples = 0.25;
meanFunction = MeanZero();
kernelFunction = SE(log(5.0),log(1.0));
noise = -1.;

In [202]:
root = GPSumNode(nextID(), Int[]);

for k in 1:K
    
    # make random (1D) split
    split = rand() * maximum(X) + minimum(X)
    n = min(sum(X .>= split), sum(X .< split))
    c = 0
    
    while n < (N * numSamples)
        @assert c < 100 "Could not find a split"
        split = rand() * maximum(X) + minimum(X)
        n = min(sum(X .>= split), sum(X .< split))
        c += 1
    end
    
    child = FiniteSplitNode(nextID(), Float64[split])
    
    # append two GP leaves
    leaf1 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .<= split], 1, sum(X .<= split)), y[X .<= split], meanFunction, kernelFunction, noise))
    leaf1.parents = SPNNode[]
    
    leaf2 = GPLeaf{Any}(nextID(),
            GP(reshape(X[X .> split], 1, sum(X .> split)), y[X .> split], meanFunction, kernelFunction, noise))
    leaf2.parents = SPNNode[]
    
    add!(child, leaf1)
    add!(child, leaf2)
    
    add!(root, child)
end

### some helper functions

In [203]:
function getAllSplits(spn)

    splitNodes = filter(n -> isa(n, FiniteSplitNode), SumProductNetworks.order(spn))
    allSplits = Dict{Int, Vector{Vector{Float64}}}()

    for splitNode in splitNodes
        d = depth(splitNode)
        if !haskey(allSplits, d)
            allSplits[d] = Vector{Vector{Float64}}(0)
        end

        push!(allSplits[d], splitNode.split)    
    end
    
    return allSplits
end

getAllSplits (generic function with 1 method)

In [204]:
splits = getAllSplits(root)

Dict{Int64,Array{Array{Float64,1},1}} with 1 entry:
  1 => Array{Float64,1}[[20.3374], [19.5447], [30.2027], [35.1944], [29.1312], …

In [205]:
function plotSplits!(plt, splits)
    depths = sort(collect(keys(splits)))
    for d in depths
        vline!(plt, [s[1] for s in splits[d]], label = "depth $(d) splits")
    end
    plt
end

plotSplits! (generic function with 1 method)

In [207]:
plt = scatter(X, y, label = "observations", title = "Random splits of SPN-GP")
plotSplits!(plt, splits)
savefig("../plots/randomSplits.png")

## Compute likelihoods

In [208]:
function spn_likelihood(node::GPLeaf)
    return node.gp.target
end

spn_likelihood (generic function with 3 methods)

In [209]:
# test
n = GPLeaf{Any}(0, GP(reshape(X, 1, N), y, meanFunction, kernelFunction, noise))
spn_likelihood(n)

-41.1350265676093

In [210]:
function spn_likelihood(node::FiniteSplitNode)
    return sum(spn_likelihood(child) for child in children(node))
end

spn_likelihood (generic function with 3 methods)

In [211]:
# test
n = root.children[3]
spn_likelihood(n)

-42.28561685154945

In [212]:
function spn_likelihood(node::GPSumNode)
    logw = log.(node.prior_weights)
    return logsumexp(logw + [spn_likelihood(child) for child in children(node)])
end

spn_likelihood (generic function with 3 methods)

In [213]:
# test
spn_likelihood(root)

-42.73473978487567

## update posterior weights

In [214]:
function spn_update(node::GPLeaf)
    return spn_likelihood(node)
end

spn_update (generic function with 3 methods)

In [215]:
function spn_update(node::FiniteSplitNode)
    return sum(spn_update(child) for child in children(node))
end

spn_update (generic function with 3 methods)

In [216]:
function spn_update(node::GPSumNode)
    logw_prior = log.(node.prior_weights)
    logw_posterior = logw_prior + [spn_update(child) for child in children(node)]
    Z = logsumexp(logw_posterior)
    node.posterior_weights = exp.(logw_posterior - Z)
    return Z
end

spn_update (generic function with 3 methods)

In [217]:
spn_update(root)

-42.73473978487567

## Functions for making predictions under the model

Leaf

In [218]:
function spn_predict(node::GPLeaf, x)
    return predict_y(node.gp, x)
end

spn_predict (generic function with 3 methods)

In [219]:
# test
n = GPLeaf{Any}(0, GP(reshape(X, 1, N), y, meanFunction, kernelFunction, noise))
μ, σ2 = spn_predict(n, linspace(0, 5, 100));

Product

In [220]:
function spn_predict(node::FiniteSplitNode, x)
    
    s = x .<= node.split
    
    μ = zeros(length(x))
    σ2 = zeros(length(x))
    
    pred1 = spn_predict(children(node)[1], x[s])
    pred2 = spn_predict(children(node)[2], x[.!s])
    
    μ[s] = pred1[1]
    μ[.!s] = pred2[1]
    
    σ2[s] = pred1[2]
    σ2[.!s] = pred2[2]
    
    
    return (μ, σ2)
end

spn_predict (generic function with 3 methods)

In [223]:
# test
n = root.children[3]
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title ="Prediction of product node 3 only")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "MoE GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/productNode3.png")

Prediction under a sum node in the SPN:
$$ \mu = \sum_k w_k \mu_k $$
$$ \sigma^2 = \sum_k w_k \sigma^2_k + \sum_k w_k \mu^2_k - (\mu)^2 $$

In [224]:
function spn_predict(node::GPSumNode, x)
    childPredictions = [spn_predict(child, x) for child in children(node)]
    
    μs = [pred[1] for pred in childPredictions]
    σ2s = [pred[2] for pred in childPredictions]
    
    μ = zeros(length(x))
    σ2 = zeros(length(x))
    
    for k in 1:length(node)
       μ += μs[k] .* node.posterior_weights[k]
       σ2 += σ2s[k] .* node.posterior_weights[k]
       σ2 += μs[k].^2 .* node.posterior_weights[k]
    end
    
    σ2 .-= μ.^2
    
    return (μ, σ2)
end

spn_predict (generic function with 3 methods)

In [225]:
@assert sum(root.prior_weights) ≈ 1.
@assert sum(root.posterior_weights) ≈ 1.

In [229]:
# test
n = root
μ, σ² = spn_predict(n, linspace(minimum(X),maximum(X),100));

plt = scatter(X, y, label = "observations", title="SPN-GP with $(K) splits and LLH: $(spn_likelihood(root))")
plot!(plt, linspace(minimum(X),maximum(X),100), μ, ribbon=2*sqrt.(σ²), label = "SPN-GP with 95% interval")
plotSplits!(plt, getAllSplits(n))
savefig("../plots/spn_gp.png")

In [241]:
vline([1, 2, 3], linewidth=(10))


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 y1

In [ ]:
function plotSplits!(plt, splits, weights)
    depths = sort(collect(keys(splits)))
    for d in depths
        vline!(plt, [s[1] for s in splits[d]], label = "depth $(d) splits")
    end
    plt
end